### Libraries and Definitions

In [1]:
import time
from selenium import webdriver
from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException

import mysql.connector

import pandas as pd
import streamlit as st

# Fucntion to launch RedBus ->>
def Launch_RedBus():
    driver = webdriver.Chrome()
    driver.maximize_window()
    driver.get('https://www.redbus.in/')
    time.sleep(5)
    return driver

# Function for Scrolling the page down ->>
def scroll_page(driver):
    body = driver.find_element(By.TAG_NAME, 'body')
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(2)

# Function for Clicking RTC Card ->>
def RTC_Card(driver,i):
    RTC_Cards = driver.find_elements(By.XPATH, "//div[@class='rtcCards']")
    for card in RTC_Cards[i:]:
        driver.execute_script("arguments[0].click();", card)
        time.sleep(4)
        break

# For Fetching Bus Route Names & Bus Route Links ->>
def Links_Routes(driver, scroll_page, Bus_Routes_Links,Bus_Routes_Names, pages):
    c=1
    for i in pages:
        if c !=1:
            driver.execute_script("arguments[0].click();", i)
        Bus_Routes = driver.find_elements(By.XPATH, "//div[@class='D117_main D117_container']//a[contains(@href, '/bus-tickets/')]")
        for j in Bus_Routes:
            Bus_Routes_Links.append(j.get_attribute('href'))
            Bus_Routes_Names.append(j.text)
        if(c==1):
            for k in range(2):
                scroll_page(driver)
            time.sleep(2)
        else:
            time.sleep(2)
        c=2


### APSRTC Details

In [4]:
# APSRTC BUS DETAILS
APSRTC_Bus_Routes_Links = []
APSRTC_Bus_Routes_Names = []

# APSRTC_ Bus Details Lists ->>
APSRTC_Bus_Routes_Links_Final = []
APSRTC_Bus_Routes_Names_Final = []
APSRTC_Bus_Names_Final = []
APSRTC_Bus_type_Final = []
APSRTC_Bus_departure_time_Final = []
APSRTC_Bus_reaching_time_Final = []
APSRTC_Bus_duration_Final = []
APSRTC_Bus_Rating_Final = []
APSRTC_Bus_seat_availability_Final = []
APSRTC_Bus_Fare_Final = []

#Launching RedBus application ->>
driver = Launch_RedBus()

#Scrolling the page down ->>
scroll_page(driver)

# Function for Clicking APSRTC Card ->>
RTC_Card(driver,0)

# APSRTC Bus Route Links and Names ->>

# Finding Page Numbers ->>
pages = driver.find_elements(By.XPATH, "//div[contains(@class, 'DC_117_pageTabs ')]")

# For Fetching Bus Route Names & Bus Route Links of APSRTC buses->>
Links_Routes(driver, scroll_page,APSRTC_Bus_Routes_Links,APSRTC_Bus_Routes_Names, pages)

for link, name in zip(APSRTC_Bus_Routes_Links, APSRTC_Bus_Routes_Names):
    try:
        driver.get(link)
        time.sleep(5)

        # Click 'View Buses' button if present
        view_buses_buttons = driver.find_elements(By.XPATH, "//div[@class='button']")
        if view_buses_buttons:
            for button in view_buses_buttons:
                try:
                    driver.execute_script("arguments[0].scrollIntoView(true);", button)
                    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//div[@class='button']")))
                    driver.execute_script("arguments[0].click();", button)
                    time.sleep(8)
                except Exception as e:
                    print(f"Exception occurred while clicking 'View Buses' button: {e}")
                    
        # Scroll to the bottom of the page
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        # Scroll back to the top of the page ->>
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(5)

        # Fetch bus details
        bus_names = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
        bus_types = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        bus_departure_times = driver.find_elements(By.XPATH, "//div[@class='dp-time f-19 d-color f-bold']")
        bus_reaching_times = driver.find_elements(By.XPATH, "//div[@class='bp-time f-19 d-color disp-Inline']")
        bus_durations = driver.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
        bus_ratings = driver.find_elements(By.XPATH, "//div[contains(@class, 'rating-sec')]")
        bus_seat_availabilities = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")
        bus_fares = driver.find_elements(By.XPATH, "//div[@class='fare d-block']")

        # Append bus details to lists
        for i in range(len(bus_names)):
            APSRTC_Bus_Routes_Links_Final.append(link)
            APSRTC_Bus_Routes_Names_Final.append(name)

        APSRTC_Bus_Names_Final.extend([bus.text for bus in bus_names])
        APSRTC_Bus_type_Final.extend([bus_type.text for bus_type in bus_types])
        APSRTC_Bus_departure_time_Final.extend([dep_time.text for dep_time in bus_departure_times])
        APSRTC_Bus_reaching_time_Final.extend([reach_time.text for reach_time in bus_reaching_times])
        APSRTC_Bus_duration_Final.extend([duration.text for duration in bus_durations])
        APSRTC_Bus_Rating_Final.extend([rating.text for rating in bus_ratings])
        APSRTC_Bus_seat_availability_Final.extend([availability.text for availability in bus_seat_availabilities])
        APSRTC_Bus_Fare_Final.extend([fare.text for fare in bus_fares])

    except Exception as e:
        print(f"Exception occurred while scraping route: {name}, {e}")

driver.quit()

# Define the CSV file name
csv_file = "csv/APSRTC_Bus_Details.csv"

# Open the CSV file in write mode
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)

    # Write the data to the CSV file
    for x,y,a,b,c,d,e,f,g,h in zip(APSRTC_Bus_Routes_Links_Final,APSRTC_Bus_Routes_Names_Final,APSRTC_Bus_Names_Final,APSRTC_Bus_type_Final,APSRTC_Bus_departure_time_Final,APSRTC_Bus_duration_Final,APSRTC_Bus_reaching_time_Final,APSRTC_Bus_Rating_Final,APSRTC_Bus_seat_availability_Final,APSRTC_Bus_Fare_Final):
        writer.writerow([x,y,a,b,c,d,e,f,g,h])

print(f"Data successfully written to {csv_file}")

TimeoutException: 'View Busses' button not found for route: Madanapalli to Bangalore
Data successfully written to csv/APSRTC_Bus_Details.csv


### KERALA_RTC Details

In [3]:
# Kerala_RTC BUS DETAILS
Kerala_RTC_Bus_Routes_Links = []
Kerala_RTC_Bus_Routes_Names = []

# Kerala_RTC_ Bus Details Lists ->>
Kerala_RTC_Bus_Routes_Links_Final = []
Kerala_RTC_Bus_Routes_Names_Final = []
Kerala_RTC_Bus_Names_Final = []
Kerala_RTC_Bus_type_Final = []
Kerala_RTC_Bus_departure_time_Final = []
Kerala_RTC_Bus_reaching_time_Final = []
Kerala_RTC_Bus_duration_Final = []
Kerala_RTC_Bus_Rating_Final = []
Kerala_RTC_Bus_seat_availability_Final = []
Kerala_RTC_Bus_Fare_Final = []

#Launching RedBus application ->>
driver = Launch_RedBus()

#Scrolling the page down ->>
scroll_page(driver)

# Function for Clicking Kerala_RTC Card ->>
RTC_Card(driver,1)

# Kerala_RTC Bus Route Links and Names ->>

# Finding Page Numbers ->>
pages = driver.find_elements(By.XPATH, "//div[contains(@class, 'DC_117_pageTabs ')]")

# For Fetching Bus Route Names & Bus Route Links of Kerala_RTC buses->>
Links_Routes(driver, scroll_page,Kerala_RTC_Bus_Routes_Links,Kerala_RTC_Bus_Routes_Names, pages)

for link, name in zip(Kerala_RTC_Bus_Routes_Links, Kerala_RTC_Bus_Routes_Names):
    try:
        driver.get(link)
        time.sleep(5)

        # Click 'View Buses' button if present
        view_buses_buttons = driver.find_elements(By.XPATH, "//div[@class='button']")
        if view_buses_buttons:
            for button in view_buses_buttons:
                try:
                    driver.execute_script("arguments[0].scrollIntoView(true);", button)
                    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//div[@class='button']")))
                    driver.execute_script("arguments[0].click();", button)
                    time.sleep(8)
                except Exception as e:
                    print(f"Exception occurred while clicking 'View Buses' button: {e}")
                    
        # Scroll to the bottom of the page
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        # Scroll back to the top of the page ->>
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(5)

        # Fetch bus details
        bus_names = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
        bus_types = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        bus_departure_times = driver.find_elements(By.XPATH, "//div[@class='dp-time f-19 d-color f-bold']")
        bus_reaching_times = driver.find_elements(By.XPATH, "//div[@class='bp-time f-19 d-color disp-Inline']")
        bus_durations = driver.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
        bus_ratings = driver.find_elements(By.XPATH, "//div[contains(@class, 'rating-sec')]")
        bus_seat_availabilities = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")
        bus_fares = driver.find_elements(By.XPATH, "//div[@class='fare d-block']")

        # Append bus details to lists
        for i in range(len(bus_names)):
            Kerala_RTC_Bus_Routes_Links_Final.append(link)
            Kerala_RTC_Bus_Routes_Names_Final.append(name)

        Kerala_RTC_Bus_Names_Final.extend([bus.text for bus in bus_names])
        Kerala_RTC_Bus_type_Final.extend([bus_type.text for bus_type in bus_types])
        Kerala_RTC_Bus_departure_time_Final.extend([dep_time.text for dep_time in bus_departure_times])
        Kerala_RTC_Bus_reaching_time_Final.extend([reach_time.text for reach_time in bus_reaching_times])
        Kerala_RTC_Bus_duration_Final.extend([duration.text for duration in bus_durations])
        Kerala_RTC_Bus_Rating_Final.extend([rating.text for rating in bus_ratings])
        Kerala_RTC_Bus_seat_availability_Final.extend([availability.text for availability in bus_seat_availabilities])
        Kerala_RTC_Bus_Fare_Final.extend([fare.text for fare in bus_fares])

    except Exception as e:
        print(f"Exception occurred while scraping route: {name}, {e}")

driver.quit()

# Define the CSV file name
csv_file = "csv/Kerala_RTC_Bus_Details.csv"

# Open the CSV file in write mode
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)

    # Write the data to the CSV file
    for x,y,a,b,c,d,e,f,g,h in zip(Kerala_RTC_Bus_Routes_Links_Final,Kerala_RTC_Bus_Routes_Names_Final,Kerala_RTC_Bus_Names_Final,Kerala_RTC_Bus_type_Final,Kerala_RTC_Bus_departure_time_Final,Kerala_RTC_Bus_duration_Final,Kerala_RTC_Bus_reaching_time_Final,Kerala_RTC_Bus_Rating_Final,Kerala_RTC_Bus_seat_availability_Final,Kerala_RTC_Bus_Fare_Final):
        writer.writerow([x,y,a,b,c,d,e,f,g,h])

print(f"Data successfully written to {csv_file}")

TimeoutException: 'View Busses' button not found for route: Coimbatore to Ooty
Data successfully written to csv/Kerala_RTC_Bus_Details.csv


### TSRTC Details

In [3]:
# TSRTC BUS DETAILS
TSRTC_Bus_Routes_Links = []
TSRTC_Bus_Routes_Names = []

# TSRTC_ Bus Details Lists ->>
TSRTC_Bus_Routes_Links_Final = []
TSRTC_Bus_Routes_Names_Final = []
TSRTC_Bus_Names_Final = []
TSRTC_Bus_type_Final = []
TSRTC_Bus_departure_time_Final = []
TSRTC_Bus_reaching_time_Final = []
TSRTC_Bus_duration_Final = []
TSRTC_Bus_Rating_Final = []
TSRTC_Bus_seat_availability_Final = []
TSRTC_Bus_Fare_Final = []

#Launching RedBus application ->>
driver = Launch_RedBus()

#Scrolling the page down ->>
scroll_page(driver)

# Function for Clicking TSRTC Card ->>
RTC_Card(driver,1)

# TSRTC Bus Route Links and Names ->>

# Finding Page Numbers ->>
pages = driver.find_elements(By.XPATH, "//div[contains(@class, 'DC_117_pageTabs ')]")

# For Fetching Bus Route Names & Bus Route Links of TSRTC buses->>
Links_Routes(driver, scroll_page,TSRTC_Bus_Routes_Links,TSRTC_Bus_Routes_Names, pages)

for link, name in zip(TSRTC_Bus_Routes_Links, TSRTC_Bus_Routes_Names):
    try:
        driver.get(link)
        time.sleep(5)

        # Click 'View Buses' button if present
        view_buses_buttons = driver.find_elements(By.XPATH, "//div[@class='button']")
        if view_buses_buttons:
            for button in view_buses_buttons:
                try:
                    driver.execute_script("arguments[0].scrollIntoView(true);", button)
                    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//div[@class='button']")))
                    driver.execute_script("arguments[0].click();", button)
                    time.sleep(8)
                except Exception as e:
                    print(f"Exception occurred while clicking 'View Buses' button: {e}")
                    
        # Scroll to the bottom of the page
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        # Scroll back to the top of the page ->>
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(5)

        # Fetch bus details
        bus_names = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
        bus_types = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        bus_departure_times = driver.find_elements(By.XPATH, "//div[@class='dp-time f-19 d-color f-bold']")
        bus_reaching_times = driver.find_elements(By.XPATH, "//div[@class='bp-time f-19 d-color disp-Inline']")
        bus_durations = driver.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
        bus_ratings = driver.find_elements(By.XPATH, "//div[contains(@class, 'rating-sec')]")
        bus_seat_availabilities = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")
        bus_fares = driver.find_elements(By.XPATH, "//div[@class='fare d-block']")

        # Append bus details to lists
        for i in range(len(bus_names)):
            TSRTC_Bus_Routes_Links_Final.append(link)
            TSRTC_Bus_Routes_Names_Final.append(name)

        TSRTC_Bus_Names_Final.extend([bus.text for bus in bus_names])
        TSRTC_Bus_type_Final.extend([bus_type.text for bus_type in bus_types])
        TSRTC_Bus_departure_time_Final.extend([dep_time.text for dep_time in bus_departure_times])
        TSRTC_Bus_reaching_time_Final.extend([reach_time.text for reach_time in bus_reaching_times])
        TSRTC_Bus_duration_Final.extend([duration.text for duration in bus_durations])
        TSRTC_Bus_Rating_Final.extend([rating.text for rating in bus_ratings])
        TSRTC_Bus_seat_availability_Final.extend([availability.text for availability in bus_seat_availabilities])
        TSRTC_Bus_Fare_Final.extend([fare.text for fare in bus_fares])

    except Exception as e:
        print(f"Exception occurred while scraping route: {name}, {e}")

driver.quit()

# Define the CSV file name
csv_file = "csv/TSRTC_Bus_Details.csv"

# Open the CSV file in write mode
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)

    # Write the data to the CSV file
    for x,y,a,b,c,d,e,f,g,h in zip(TSRTC_Bus_Routes_Links_Final,TSRTC_Bus_Routes_Names_Final,TSRTC_Bus_Names_Final,TSRTC_Bus_type_Final,TSRTC_Bus_departure_time_Final,TSRTC_Bus_duration_Final,TSRTC_Bus_reaching_time_Final,TSRTC_Bus_Rating_Final,TSRTC_Bus_seat_availability_Final,TSRTC_Bus_Fare_Final):
        writer.writerow([x,y,a,b,c,d,e,f,g,h])

print(f"Data successfully written to {csv_file}")

TimeoutException: 'View Busses' button not found for route: Hyderabad to Godavarikhani
TimeoutException: 'View Busses' button not found for route: Hyderabad to Warangal
Data successfully written to csv/TSRTC_Bus_Details.csv


### KTCL Details

In [2]:
# KTCL BUS DETAILS
KTCL_Bus_Routes_Links = []
KTCL_Bus_Routes_Names = []

# KTCL_ Bus Details Lists ->>
KTCL_Bus_Routes_Links_Final = []
KTCL_Bus_Routes_Names_Final = []
KTCL_Bus_Names_Final = []
KTCL_Bus_type_Final = []
KTCL_Bus_departure_time_Final = []
KTCL_Bus_reaching_time_Final = []
KTCL_Bus_duration_Final = []
KTCL_Bus_Rating_Final = []
KTCL_Bus_seat_availability_Final = []
KTCL_Bus_Fare_Final = []

#Launching RedBus application ->>
driver = Launch_RedBus()

#Scrolling the page down ->>
scroll_page(driver)

# Function for Clicking KTCL Card ->>
RTC_Card(driver,3)

# KTCL Bus Route Links and Names ->>

# Finding Page Numbers ->>
pages = driver.find_elements(By.XPATH, "//div[contains(@class, 'DC_117_pageTabs ')]")

# For Fetching Bus Route Names & Bus Route Links of KTCL buses->>
Links_Routes(driver, scroll_page,KTCL_Bus_Routes_Links,KTCL_Bus_Routes_Names, pages)

for link, name in zip(KTCL_Bus_Routes_Links, KTCL_Bus_Routes_Names):
    try:
        driver.get(link)
        time.sleep(5)

        # Click 'View Buses' button if present
        view_buses_buttons = driver.find_elements(By.XPATH, "//div[@class='button']")
        if view_buses_buttons:
            for button in view_buses_buttons:
                try:
                    driver.execute_script("arguments[0].scrollIntoView(true);", button)
                    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//div[@class='button']")))
                    driver.execute_script("arguments[0].click();", button)
                    time.sleep(8)
                except Exception as e:
                    print(f"Exception occurred while clicking 'View Buses' button: {e}")
                    
        # Scroll to the bottom of the page
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        # Scroll back to the top of the page ->>
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(5)

        # Fetch bus details
        bus_names = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
        bus_types = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        bus_departure_times = driver.find_elements(By.XPATH, "//div[@class='dp-time f-19 d-color f-bold']")
        bus_reaching_times = driver.find_elements(By.XPATH, "//div[@class='bp-time f-19 d-color disp-Inline']")
        bus_durations = driver.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
        bus_ratings = driver.find_elements(By.XPATH, "//div[contains(@class, 'rating-sec')]")
        bus_seat_availabilities = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")
        bus_fares = driver.find_elements(By.XPATH, "//div[@class='fare d-block']")

        # Append bus details to lists
        for i in range(len(bus_names)):
            KTCL_Bus_Routes_Links_Final.append(link)
            KTCL_Bus_Routes_Names_Final.append(name)

        KTCL_Bus_Names_Final.extend([bus.text for bus in bus_names])
        KTCL_Bus_type_Final.extend([bus_type.text for bus_type in bus_types])
        KTCL_Bus_departure_time_Final.extend([dep_time.text for dep_time in bus_departure_times])
        KTCL_Bus_reaching_time_Final.extend([reach_time.text for reach_time in bus_reaching_times])
        KTCL_Bus_duration_Final.extend([duration.text for duration in bus_durations])
        KTCL_Bus_Rating_Final.extend([rating.text for rating in bus_ratings])
        KTCL_Bus_seat_availability_Final.extend([availability.text for availability in bus_seat_availabilities])
        KTCL_Bus_Fare_Final.extend([fare.text for fare in bus_fares])

    except Exception as e:
        print(f"Exception occurred while scraping route: {name}, {e}")

driver.quit()

# Define the CSV file name
csv_file = "csv/KTCL_Bus_Details.csv"

# Open the CSV file in write mode
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)

    # Write the data to the CSV file
    for x,y,a,b,c,d,e,f,g,h in zip(KTCL_Bus_Routes_Links_Final,KTCL_Bus_Routes_Names_Final,KTCL_Bus_Names_Final,KTCL_Bus_type_Final,KTCL_Bus_departure_time_Final,KTCL_Bus_duration_Final,KTCL_Bus_reaching_time_Final,KTCL_Bus_Rating_Final,KTCL_Bus_seat_availability_Final,KTCL_Bus_Fare_Final):
        writer.writerow([x,y,a,b,c,d,e,f,g,h])

print(f"Data successfully written to {csv_file}")

Data successfully written to csv/KTCL_Bus_Details.csv


### RSRTC Details

In [2]:
# RSRTC BUS DETAILS
RSRTC_Bus_Routes_Links = []
RSRTC_Bus_Routes_Names = []

# RSRTC_ Bus Details Lists ->>
RSRTC_Bus_Routes_Links_Final = []
RSRTC_Bus_Routes_Names_Final = []
RSRTC_Bus_Names_Final = []
RSRTC_Bus_type_Final = []
RSRTC_Bus_departure_time_Final = []
RSRTC_Bus_reaching_time_Final = []
RSRTC_Bus_duration_Final = []
RSRTC_Bus_Rating_Final = []
RSRTC_Bus_seat_availability_Final = []
RSRTC_Bus_Fare_Final = []

#Launching RedBus application ->>
driver = Launch_RedBus()

#Scrolling the page down ->>
scroll_page(driver)

# Function for Clicking RSRTC Card ->>
RTC_Card(driver,4)

# RSRTC Bus Route Links and Names ->>

# Finding Page Numbers ->>
pages = driver.find_elements(By.XPATH, "//div[contains(@class, 'DC_117_pageTabs ')]")

# For Fetching Bus Route Names & Bus Route Links of RSRTC buses->>
Links_Routes(driver, scroll_page,RSRTC_Bus_Routes_Links,RSRTC_Bus_Routes_Names, pages)

for link, name in zip(RSRTC_Bus_Routes_Links, RSRTC_Bus_Routes_Names):
    try:
        driver.get(link)
        time.sleep(5)

        # Click 'View Buses' button if present
        view_buses_buttons = driver.find_elements(By.XPATH, "//div[@class='button']")
        if view_buses_buttons:
            for button in view_buses_buttons:
                try:
                    driver.execute_script("arguments[0].scrollIntoView(true);", button)
                    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//div[@class='button']")))
                    driver.execute_script("arguments[0].click();", button)
                    time.sleep(8)
                except Exception as e:
                    print(f"Exception occurred while clicking 'View Buses' button: {e}")
                    
        # Scroll to the bottom of the page
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        # Scroll back to the top of the page ->>
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(5)

        # Fetch bus details
        bus_names = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
        bus_types = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        bus_departure_times = driver.find_elements(By.XPATH, "//div[@class='dp-time f-19 d-color f-bold']")
        bus_reaching_times = driver.find_elements(By.XPATH, "//div[@class='bp-time f-19 d-color disp-Inline']")
        bus_durations = driver.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
        bus_ratings = driver.find_elements(By.XPATH, "//div[contains(@class, 'rating-sec')]")
        bus_seat_availabilities = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")
        bus_fares = driver.find_elements(By.XPATH, "//div[@class='fare d-block']")

        # Append bus details to lists
        for i in range(len(bus_names)):
            RSRTC_Bus_Routes_Links_Final.append(link)
            RSRTC_Bus_Routes_Names_Final.append(name)

        RSRTC_Bus_Names_Final.extend([bus.text for bus in bus_names])
        RSRTC_Bus_type_Final.extend([bus_type.text for bus_type in bus_types])
        RSRTC_Bus_departure_time_Final.extend([dep_time.text for dep_time in bus_departure_times])
        RSRTC_Bus_reaching_time_Final.extend([reach_time.text for reach_time in bus_reaching_times])
        RSRTC_Bus_duration_Final.extend([duration.text for duration in bus_durations])
        RSRTC_Bus_Rating_Final.extend([rating.text for rating in bus_ratings])
        RSRTC_Bus_seat_availability_Final.extend([availability.text for availability in bus_seat_availabilities])
        RSRTC_Bus_Fare_Final.extend([fare.text for fare in bus_fares])

    except Exception as e:
        print(f"Exception occurred while scraping route: {name}, {e}")

driver.quit()

# Define the CSV file name
csv_file = "csv/RSRTC_Bus_Details.csv"

# Open the CSV file in write mode
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)

    # Write the data to the CSV file
    for x,y,a,b,c,d,e,f,g,h in zip(RSRTC_Bus_Routes_Links_Final,RSRTC_Bus_Routes_Names_Final,RSRTC_Bus_Names_Final,RSRTC_Bus_type_Final,RSRTC_Bus_departure_time_Final,RSRTC_Bus_duration_Final,RSRTC_Bus_reaching_time_Final,RSRTC_Bus_Rating_Final,RSRTC_Bus_seat_availability_Final,RSRTC_Bus_Fare_Final):
        writer.writerow([x,y,a,b,c,d,e,f,g,h])

print(f"Data successfully written to {csv_file}")

Data successfully written to csv/RSRTC_Bus_Details.csv


### SBSTC Details

In [2]:
# SBSTC BUS DETAILS
SBSTC_Bus_Routes_Links = []
SBSTC_Bus_Routes_Names = []

# SBSTC_ Bus Details Lists ->>
SBSTC_Bus_Routes_Links_Final = []
SBSTC_Bus_Routes_Names_Final = []
SBSTC_Bus_Names_Final = []
SBSTC_Bus_type_Final = []
SBSTC_Bus_departure_time_Final = []
SBSTC_Bus_reaching_time_Final = []
SBSTC_Bus_duration_Final = []
SBSTC_Bus_Rating_Final = []
SBSTC_Bus_seat_availability_Final = []
SBSTC_Bus_Fare_Final = []

#Launching RedBus application ->>
driver = Launch_RedBus()

#Scrolling the page down ->>
scroll_page(driver)

# Function for Clicking SBSTC Card ->>
RTC_Card(driver,5)

# SBSTC Bus Route Links and Names ->>

# Finding Page Numbers ->>
pages = driver.find_elements(By.XPATH, "//div[contains(@class, 'DC_117_pageTabs ')]")

# For Fetching Bus Route Names & Bus Route Links of SBSTC buses->>
Links_Routes(driver, scroll_page,SBSTC_Bus_Routes_Links,SBSTC_Bus_Routes_Names, pages)

for link, name in zip(SBSTC_Bus_Routes_Links, SBSTC_Bus_Routes_Names):
    try:
        driver.get(link)
        time.sleep(5)

        # Click 'View Buses' button if present
        view_buses_buttons = driver.find_elements(By.XPATH, "//div[@class='button']")
        if view_buses_buttons:
            for button in view_buses_buttons:
                try:
                    driver.execute_script("arguments[0].scrollIntoView(true);", button)
                    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//div[@class='button']")))
                    driver.execute_script("arguments[0].click();", button)
                    time.sleep(8)
                except Exception as e:
                    print(f"Exception occurred while clicking 'View Buses' button: {e}")
                    
        # Scroll to the bottom of the page
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        # Scroll back to the top of the page ->>
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(5)

        # Fetch bus details
        bus_names = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
        bus_types = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        bus_departure_times = driver.find_elements(By.XPATH, "//div[@class='dp-time f-19 d-color f-bold']")
        bus_reaching_times = driver.find_elements(By.XPATH, "//div[@class='bp-time f-19 d-color disp-Inline']")
        bus_durations = driver.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
        bus_ratings = driver.find_elements(By.XPATH, "//div[contains(@class, 'rating-sec')]")
        bus_seat_availabilities = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")
        bus_fares = driver.find_elements(By.XPATH, "//div[@class='fare d-block']")

        # Append bus details to lists
        for i in range(len(bus_names)):
            SBSTC_Bus_Routes_Links_Final.append(link)
            SBSTC_Bus_Routes_Names_Final.append(name)

        SBSTC_Bus_Names_Final.extend([bus.text for bus in bus_names])
        SBSTC_Bus_type_Final.extend([bus_type.text for bus_type in bus_types])
        SBSTC_Bus_departure_time_Final.extend([dep_time.text for dep_time in bus_departure_times])
        SBSTC_Bus_reaching_time_Final.extend([reach_time.text for reach_time in bus_reaching_times])
        SBSTC_Bus_duration_Final.extend([duration.text for duration in bus_durations])
        SBSTC_Bus_Rating_Final.extend([rating.text for rating in bus_ratings])
        SBSTC_Bus_seat_availability_Final.extend([availability.text for availability in bus_seat_availabilities])
        SBSTC_Bus_Fare_Final.extend([fare.text for fare in bus_fares])

    except Exception as e:
        print(f"Exception occurred while scraping route: {name}, {e}")

driver.quit()

# Define the CSV file name
csv_file = "csv/SBSTC_Bus_Details.csv"

# Open the CSV file in write mode
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)

    # Write the data to the CSV file
    for x,y,a,b,c,d,e,f,g,h in zip(SBSTC_Bus_Routes_Links_Final,SBSTC_Bus_Routes_Names_Final,SBSTC_Bus_Names_Final,SBSTC_Bus_type_Final,SBSTC_Bus_departure_time_Final,SBSTC_Bus_duration_Final,SBSTC_Bus_reaching_time_Final,SBSTC_Bus_Rating_Final,SBSTC_Bus_seat_availability_Final,SBSTC_Bus_Fare_Final):
        writer.writerow([x,y,a,b,c,d,e,f,g,h])

print(f"Data successfully written to {csv_file}")

Data successfully written to csv/SBSTC_Bus_Details.csv


### HRTC Details

In [2]:
# HRTC BUS DETAILS
HRTC_Bus_Routes_Links = []
HRTC_Bus_Routes_Names = []

# HRTC_ Bus Details Lists ->>
HRTC_Bus_Routes_Links_Final = []
HRTC_Bus_Routes_Names_Final = []
HRTC_Bus_Names_Final = []
HRTC_Bus_type_Final = []
HRTC_Bus_departure_time_Final = []
HRTC_Bus_reaching_time_Final = []
HRTC_Bus_duration_Final = []
HRTC_Bus_Rating_Final = []
HRTC_Bus_seat_availability_Final = []
HRTC_Bus_Fare_Final = []

#Launching RedBus application ->>
driver = Launch_RedBus()

#Scrolling the page down ->>
scroll_page(driver)

# Function for Clicking HRTC Card ->>
RTC_Card(driver,6)

# HRTC Bus Route Links and Names ->>

# Finding Page Numbers ->>
pages = driver.find_elements(By.XPATH, "//div[contains(@class, 'DC_117_pageTabs ')]")

# For Fetching Bus Route Names & Bus Route Links of HRTC buses->>
Links_Routes(driver, scroll_page,HRTC_Bus_Routes_Links,HRTC_Bus_Routes_Names, pages)

for link, name in zip(HRTC_Bus_Routes_Links, HRTC_Bus_Routes_Names):
    try:
        driver.get(link)
        time.sleep(5)

        # Click 'View Buses' button if present
        view_buses_buttons = driver.find_elements(By.XPATH, "//div[@class='button']")
        if view_buses_buttons:
            for button in view_buses_buttons:
                try:
                    driver.execute_script("arguments[0].scrollIntoView(true);", button)
                    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//div[@class='button']")))
                    driver.execute_script("arguments[0].click();", button)
                    time.sleep(8)
                except Exception as e:
                    print(f"Exception occurred while clicking 'View Buses' button: {e}")
                    
        # Scroll to the bottom of the page
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        # Scroll back to the top of the page ->>
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(5)

        # Fetch bus details
        bus_names = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
        bus_types = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        bus_departure_times = driver.find_elements(By.XPATH, "//div[@class='dp-time f-19 d-color f-bold']")
        bus_reaching_times = driver.find_elements(By.XPATH, "//div[@class='bp-time f-19 d-color disp-Inline']")
        bus_durations = driver.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
        bus_ratings = driver.find_elements(By.XPATH, "//div[contains(@class, 'rating-sec')]")
        bus_seat_availabilities = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")
        bus_fares = driver.find_elements(By.XPATH, "//div[@class='fare d-block']")

        # Append bus details to lists
        for i in range(len(bus_names)):
            HRTC_Bus_Routes_Links_Final.append(link)
            HRTC_Bus_Routes_Names_Final.append(name)

        HRTC_Bus_Names_Final.extend([bus.text for bus in bus_names])
        HRTC_Bus_type_Final.extend([bus_type.text for bus_type in bus_types])
        HRTC_Bus_departure_time_Final.extend([dep_time.text for dep_time in bus_departure_times])
        HRTC_Bus_reaching_time_Final.extend([reach_time.text for reach_time in bus_reaching_times])
        HRTC_Bus_duration_Final.extend([duration.text for duration in bus_durations])
        HRTC_Bus_Rating_Final.extend([rating.text for rating in bus_ratings])
        HRTC_Bus_seat_availability_Final.extend([availability.text for availability in bus_seat_availabilities])
        HRTC_Bus_Fare_Final.extend([fare.text for fare in bus_fares])

    except Exception as e:
        print(f"Exception occurred while scraping route: {name}, {e}")

driver.quit()

# Define the CSV file name
csv_file = "csv/HRTC_Bus_Details.csv"

# Open the CSV file in write mode
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)

    # Write the data to the CSV file
    for x,y,a,b,c,d,e,f,g,h in zip(HRTC_Bus_Routes_Links_Final,HRTC_Bus_Routes_Names_Final,HRTC_Bus_Names_Final,HRTC_Bus_type_Final,HRTC_Bus_departure_time_Final,HRTC_Bus_duration_Final,HRTC_Bus_reaching_time_Final,HRTC_Bus_Rating_Final,HRTC_Bus_seat_availability_Final,HRTC_Bus_Fare_Final):
        writer.writerow([x,y,a,b,c,d,e,f,g,h])

print(f"Data successfully written to {csv_file}")

Data successfully written to csv/HRTC_Bus_Details.csv


### ASTC Details

In [2]:
# ASTC BUS DETAILS
ASTC_Bus_Routes_Links = []
ASTC_Bus_Routes_Names = []

# ASTC_ Bus Details Lists ->>
ASTC_Bus_Routes_Links_Final = []
ASTC_Bus_Routes_Names_Final = []
ASTC_Bus_Names_Final = []
ASTC_Bus_type_Final = []
ASTC_Bus_departure_time_Final = []
ASTC_Bus_reaching_time_Final = []
ASTC_Bus_duration_Final = []
ASTC_Bus_Rating_Final = []
ASTC_Bus_seat_availability_Final = []
ASTC_Bus_Fare_Final = []

#Launching RedBus application ->>
driver = Launch_RedBus()

#Scrolling the page down ->>
scroll_page(driver)

# Function for Clicking ASTC Card ->>
RTC_Card(driver,7)

# ASTC Bus Route Links and Names ->>

# Finding Page Numbers ->>
pages = driver.find_elements(By.XPATH, "//div[contains(@class, 'DC_117_pageTabs ')]")

# For Fetching Bus Route Names & Bus Route Links of ASTC buses->>
Links_Routes(driver, scroll_page,ASTC_Bus_Routes_Links,ASTC_Bus_Routes_Names, pages)

for link, name in zip(ASTC_Bus_Routes_Links, ASTC_Bus_Routes_Names):
    try:
        driver.get(link)
        time.sleep(5)

        # Click 'View Buses' button if present
        view_buses_buttons = driver.find_elements(By.XPATH, "//div[@class='button']")
        if view_buses_buttons:
            for button in view_buses_buttons:
                try:
                    driver.execute_script("arguments[0].scrollIntoView(true);", button)
                    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//div[@class='button']")))
                    driver.execute_script("arguments[0].click();", button)
                    time.sleep(8)
                except Exception as e:
                    print(f"Exception occurred while clicking 'View Buses' button: {e}")
                    
        # Scroll to the bottom of the page
        while True:
            # Scroll to the bottom of the page
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(4)  # Adjust sleep time as needed
            
            # Check if the element with id "seo-data" is visible
            seo_data_element = driver.find_element(By.ID,"rh_footer_new")
            if seo_data_element.is_displayed():
                break  # Exit the loop if element is found
        # Scroll back to the top of the page
        driver.execute_script("arguments[0].scrollIntoView();", seo_data_element)
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(5)

        # Fetch bus details
        bus_names = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
        bus_types = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        bus_departure_times = driver.find_elements(By.XPATH, "//div[@class='dp-time f-19 d-color f-bold']")
        bus_reaching_times = driver.find_elements(By.XPATH, "//div[@class='bp-time f-19 d-color disp-Inline']")
        bus_durations = driver.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
        bus_ratings = driver.find_elements(By.XPATH, "//div[contains(@class, 'rating-sec')]")
        bus_seat_availabilities = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")
        bus_fares = driver.find_elements(By.XPATH, "//div[@class='fare d-block']")

        # Append bus details to lists
        for i in range(len(bus_names)):
            ASTC_Bus_Routes_Links_Final.append(link)
            ASTC_Bus_Routes_Names_Final.append(name)

        ASTC_Bus_Names_Final.extend([bus.text for bus in bus_names])
        ASTC_Bus_type_Final.extend([bus_type.text for bus_type in bus_types])
        ASTC_Bus_departure_time_Final.extend([dep_time.text for dep_time in bus_departure_times])
        ASTC_Bus_reaching_time_Final.extend([reach_time.text for reach_time in bus_reaching_times])
        ASTC_Bus_duration_Final.extend([duration.text for duration in bus_durations])
        ASTC_Bus_Rating_Final.extend([rating.text for rating in bus_ratings])
        ASTC_Bus_seat_availability_Final.extend([availability.text for availability in bus_seat_availabilities])
        ASTC_Bus_Fare_Final.extend([fare.text for fare in bus_fares])

    except Exception as e:
        print(f"Exception occurred while scraping route: {name}, {e}")

driver.quit()

# Define the CSV file name
csv_file = "csv/ASTC_Bus_Details.csv"

# Open the CSV file in write mode
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)

    # Write the data to the CSV file
    for x,y,a,b,c,d,e,f,g,h in zip(ASTC_Bus_Routes_Links_Final,ASTC_Bus_Routes_Names_Final,ASTC_Bus_Names_Final,ASTC_Bus_type_Final,ASTC_Bus_departure_time_Final,ASTC_Bus_duration_Final,ASTC_Bus_reaching_time_Final,ASTC_Bus_Rating_Final,ASTC_Bus_seat_availability_Final,ASTC_Bus_Fare_Final):
        writer.writerow([x,y,a,b,c,d,e,f,g,h])

print(f"Data successfully written to {csv_file}")

Data successfully written to csv/ASTC_Bus_Details.csv


### UPSRTC Details

In [2]:
# UPSRTC BUS DETAILS
UPSRTC_Bus_Routes_Links = []
UPSRTC_Bus_Routes_Names = []

# UPSRTC_ Bus Details Lists ->>
UPSRTC_Bus_Routes_Links_Final = []
UPSRTC_Bus_Routes_Names_Final = []
UPSRTC_Bus_Names_Final = []
UPSRTC_Bus_type_Final = []
UPSRTC_Bus_departure_time_Final = []
UPSRTC_Bus_reaching_time_Final = []
UPSRTC_Bus_duration_Final = []
UPSRTC_Bus_Rating_Final = []
UPSRTC_Bus_seat_availability_Final = []
UPSRTC_Bus_Fare_Final = []

#Launching RedBus application ->>
driver = Launch_RedBus()

#Scrolling the page down ->>
scroll_page(driver)

# Function for Clicking UPSRTC Card ->>
RTC_Card(driver,8)

# UPSRTC Bus Route Links and Names ->>

# Finding Page Numbers ->>
pages = driver.find_elements(By.XPATH, "//div[contains(@class, 'DC_117_pageTabs ')]")

# For Fetching Bus Route Names & Bus Route Links of UPSRTC buses->>
Links_Routes(driver, scroll_page,UPSRTC_Bus_Routes_Links,UPSRTC_Bus_Routes_Names, pages)

for link, name in zip(UPSRTC_Bus_Routes_Links, UPSRTC_Bus_Routes_Names):
    try:
        driver.get(link)
        time.sleep(5)

        # Click 'View Buses' button if present
        view_buses_buttons = driver.find_elements(By.XPATH, "//div[@class='button']")
        if view_buses_buttons:
            for button in view_buses_buttons:
                try:
                    driver.execute_script("arguments[0].scrollIntoView(true);", button)
                    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//div[@class='button']")))
                    driver.execute_script("arguments[0].click();", button)
                    time.sleep(8)
                except Exception as e:
                    print(f"Exception occurred while clicking 'View Buses' button: {e}")
                    
        # Scroll to the bottom of the page
        while True:
            # Scroll to the bottom of the page
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(4)  # Adjust sleep time as needed
            
            # Check if the element with id "seo-data" is visible
            seo_data_element = driver.find_element(By.ID,"rh_footer_new")
            if seo_data_element.is_displayed():
                break  # Exit the loop if element is found
        # Scroll back to the top of the page
        driver.execute_script("arguments[0].scrollIntoView();", seo_data_element)
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(5)

        # Fetch bus details
        bus_names = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
        bus_types = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        bus_departure_times = driver.find_elements(By.XPATH, "//div[@class='dp-time f-19 d-color f-bold']")
        bus_reaching_times = driver.find_elements(By.XPATH, "//div[@class='bp-time f-19 d-color disp-Inline']")
        bus_durations = driver.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
        bus_ratings = driver.find_elements(By.XPATH, "//div[contains(@class, 'rating-sec')]")
        bus_seat_availabilities = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")
        bus_fares = driver.find_elements(By.XPATH, "//div[@class='fare d-block']")

        # Append bus details to lists
        for i in range(len(bus_names)):
            UPSRTC_Bus_Routes_Links_Final.append(link)
            UPSRTC_Bus_Routes_Names_Final.append(name)

        UPSRTC_Bus_Names_Final.extend([bus.text for bus in bus_names])
        UPSRTC_Bus_type_Final.extend([bus_type.text for bus_type in bus_types])
        UPSRTC_Bus_departure_time_Final.extend([dep_time.text for dep_time in bus_departure_times])
        UPSRTC_Bus_reaching_time_Final.extend([reach_time.text for reach_time in bus_reaching_times])
        UPSRTC_Bus_duration_Final.extend([duration.text for duration in bus_durations])
        UPSRTC_Bus_Rating_Final.extend([rating.text for rating in bus_ratings])
        UPSRTC_Bus_seat_availability_Final.extend([availability.text for availability in bus_seat_availabilities])
        UPSRTC_Bus_Fare_Final.extend([fare.text for fare in bus_fares])

    except Exception as e:
        print(f"Exception occurred while scraping route: {name}, {e}")

driver.quit()

# Define the CSV file name
csv_file = "csv/UPSRTC_Bus_Details.csv"

# Open the CSV file in write mode
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)

    # Write the data to the CSV file
    for x,y,a,b,c,d,e,f,g,h in zip(UPSRTC_Bus_Routes_Links_Final,UPSRTC_Bus_Routes_Names_Final,UPSRTC_Bus_Names_Final,UPSRTC_Bus_type_Final,UPSRTC_Bus_departure_time_Final,UPSRTC_Bus_duration_Final,UPSRTC_Bus_reaching_time_Final,UPSRTC_Bus_Rating_Final,UPSRTC_Bus_seat_availability_Final,UPSRTC_Bus_Fare_Final):
        writer.writerow([x,y,a,b,c,d,e,f,g,h])

print(f"Data successfully written to {csv_file}")

Data successfully written to csv/UPSRTC_Bus_Details.csv


### WBTC Details

In [2]:
# WBTC BUS DETAILS
WBTC_Bus_Routes_Links = []
WBTC_Bus_Routes_Names = []

# WBTC_ Bus Details Lists ->>
WBTC_Bus_Routes_Links_Final = []
WBTC_Bus_Routes_Names_Final = []
WBTC_Bus_Names_Final = []
WBTC_Bus_type_Final = []
WBTC_Bus_departure_time_Final = []
WBTC_Bus_reaching_time_Final = []
WBTC_Bus_duration_Final = []
WBTC_Bus_Rating_Final = []
WBTC_Bus_seat_availability_Final = []
WBTC_Bus_Fare_Final = []

#Launching RedBus application ->>
driver = Launch_RedBus()

#Scrolling the page down ->>
scroll_page(driver)

# Function for Clicking WBTC Card ->>
RTC_Card(driver,9)

# WBTC Bus Route Links and Names ->>

# Finding Page Numbers ->>
pages = driver.find_elements(By.XPATH, "//div[contains(@class, 'DC_117_pageTabs ')]")

# For Fetching Bus Route Names & Bus Route Links of WBTC buses->>
Links_Routes(driver, scroll_page,WBTC_Bus_Routes_Links,WBTC_Bus_Routes_Names, pages)

for link, name in zip(WBTC_Bus_Routes_Links, WBTC_Bus_Routes_Names):
    try:
        driver.get(link)
        time.sleep(5)

        # Click 'View Buses' button if present
        view_buses_buttons = driver.find_elements(By.XPATH, "//div[@class='button']")
        if view_buses_buttons:
            for button in view_buses_buttons:
                try:
                    driver.execute_script("arguments[0].scrollIntoView(true);", button)
                    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//div[@class='button']")))
                    driver.execute_script("arguments[0].click();", button)
                    time.sleep(8)
                except Exception as e:
                    print(f"Exception occurred while clicking 'View Buses' button: {e}")
                    
        # Scroll to the bottom of the page
        # Scroll to the bottom of the page in increments ->>
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        # Scroll back to the top of the page ->>
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(5)

        # Fetch bus details
        bus_names = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
        bus_types = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        bus_departure_times = driver.find_elements(By.XPATH, "//div[@class='dp-time f-19 d-color f-bold']")
        bus_reaching_times = driver.find_elements(By.XPATH, "//div[@class='bp-time f-19 d-color disp-Inline']")
        bus_durations = driver.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
        bus_ratings = driver.find_elements(By.XPATH, "//div[contains(@class, 'rating-sec')]")
        bus_seat_availabilities = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")
        bus_fares = driver.find_elements(By.XPATH, "//div[@class='fare d-block']")

        # Append bus details to lists
        for i in range(len(bus_names)):
            WBTC_Bus_Routes_Links_Final.append(link)
            WBTC_Bus_Routes_Names_Final.append(name)

        WBTC_Bus_Names_Final.extend([bus.text for bus in bus_names])
        WBTC_Bus_type_Final.extend([bus_type.text for bus_type in bus_types])
        WBTC_Bus_departure_time_Final.extend([dep_time.text for dep_time in bus_departure_times])
        WBTC_Bus_reaching_time_Final.extend([reach_time.text for reach_time in bus_reaching_times])
        WBTC_Bus_duration_Final.extend([duration.text for duration in bus_durations])
        WBTC_Bus_Rating_Final.extend([rating.text for rating in bus_ratings])
        WBTC_Bus_seat_availability_Final.extend([availability.text for availability in bus_seat_availabilities])
        WBTC_Bus_Fare_Final.extend([fare.text for fare in bus_fares])

    except Exception as e:
        print(f"Exception occurred while scraping route: {name}, {e}")

driver.quit()

# Define the CSV file name
csv_file = "csv/WBTC_Bus_Details.csv"

# Open the CSV file in write mode
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)

    # Write the data to the CSV file
    for x,y,a,b,c,d,e,f,g,h in zip(WBTC_Bus_Routes_Links_Final,WBTC_Bus_Routes_Names_Final,WBTC_Bus_Names_Final,WBTC_Bus_type_Final,WBTC_Bus_departure_time_Final,WBTC_Bus_duration_Final,WBTC_Bus_reaching_time_Final,WBTC_Bus_Rating_Final,WBTC_Bus_seat_availability_Final,WBTC_Bus_Fare_Final):
        writer.writerow([x,y,a,b,c,d,e,f,g,h])

print(f"Data successfully written to {csv_file}")

Data successfully written to csv/WBTC_Bus_Details.csv


### CSV's To DataFrame

In [2]:
#Creating DataFrame from the scrapped data

column_names = ['bus_routes_link', 'bus_routes_name', 'bus_name', 'bus_type', 'departure_time', 'duration', 'arrival_time', 'rating', 'seats_available', 'price']

data = pd.read_csv('csv/APSRTC_Bus_Details.csv', names=column_names)

# Read the second CSV file
data2 = pd.read_csv('csv/ASTC_Bus_Details.csv', names=column_names)
data = pd.concat([data, data2])

data3 = pd.read_csv('csv/HRTC_Bus_Details.csv', names=column_names)
data = pd.concat([data, data3])

data4 = pd.read_csv('csv/Kerala_RTC_Bus_Details.csv', names=column_names)
data = pd.concat([data, data4])

data5 = pd.read_csv('csv/KTCL_Bus_Details.csv', names=column_names)
data = pd.concat([data, data5])

data6 = pd.read_csv('csv/RSRTC_Bus_Details.csv', names=column_names)
data = pd.concat([data, data6])

data7 = pd.read_csv('csv/SBSTC_Bus_Details.csv', names=column_names)
data = pd.concat([data, data7])

data8 = pd.read_csv('csv/TSRTC_Bus_Details.csv', names=column_names)
data = pd.concat([data, data8])

data9 = pd.read_csv('csv/UPSRTC_Bus_Details.csv', names=column_names)
data = pd.concat([data, data9])

data10 = pd.read_csv('csv/WBTC_Bus_Details.csv', names=column_names)
data = pd.concat([data, data10])

# Display the DataFrame
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14797 entries, 0 to 1240
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   bus_routes_link  14797 non-null  object 
 1   bus_routes_name  14797 non-null  object 
 2   bus_name         14797 non-null  object 
 3   bus_type         14797 non-null  object 
 4   departure_time   14797 non-null  object 
 5   duration         14797 non-null  object 
 6   arrival_time     14797 non-null  object 
 7   rating           14797 non-null  float64
 8   seats_available  14797 non-null  object 
 9   price            14797 non-null  float64
dtypes: float64(2), object(8)
memory usage: 1.2+ MB


In [3]:
data.to_csv('bus_details.csv', header=False, index=False)

### Inserting Into DataBase

In [3]:
# Connect to MySQL
conn = mysql.connector.connect(host='127.0.0.1',database='redbus',user='root',password='admin')
cursor = conn.cursor()
create_table_query = '''
CREATE TABLE IF NOT EXISTS bus_details (
    id INT AUTO_INCREMENT PRIMARY KEY,
    bus_routes_link VARCHAR(255),
    bus_routes_name VARCHAR(255),
    bus_name VARCHAR(255),
    bus_type VARCHAR(255),
    departure_time VARCHAR(255),
    duration VARCHAR(255),
    arrival_time VARCHAR(255),
    rating FLOAT,
    seats_available VARCHAR(255), 
    price INT
)
'''

cursor.execute(create_table_query)

for index, row in data.iterrows():
    sql = '''INSERT INTO bus_details (bus_routes_link, bus_routes_name, bus_name, bus_type, departure_time, duration, arrival_time, rating, seats_available, price)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'''
    # Convert seats_available to string before insertion
    values = (row['bus_routes_link'], row['bus_routes_name'], row['bus_name'], row['bus_type'],
                row['departure_time'], row['duration'], row['arrival_time'], row['rating'],
              row['seats_available'], row['price'])
    cursor.execute(sql, values)
    conn.commit()
cursor.close()
conn.close()

